## Monte Carlo swaption pricer

Pricing swaptions using Monte-Carlo

-Able to select between random number generator/Sobol

-Results also with annuity factor as control variate instrument

In [1]:
import os
import sys

In [2]:
current_dir = os.getcwd()
qg_dir = os.path.join(current_dir, "..")

sys.path.append(qg_dir)

In [3]:
from quassigaussian.montecarlo.simulations import ProcessSimulatorTerminalMeasure
from quassigaussian.montecarlo.monte_carlo_pricer import monte_carlo_pricer_terminal_measure
from quassigaussian.parameters.volatility.local_volatility import LinearLocalVolatility
from quassigaussian.products.instruments import Swap, Swaption
from quassigaussian.montecarlo.control_variate import apply_control_variate_annuity
from quassigaussian.products.pricer import SwapPricer, SwaptionPricer, find_implied_black_vola
import pandas as pd
import os
import numpy as np
from qgtests.utis import get_mock_yield_curve_const
from report.directories import output_data_raw, date_timestamp
from report.utils import get_nonexistant_path


In [4]:
number_paths = 2**14
number_time_steps = 2**10

random_number_generator_type = "sobol"
random_number_generator_type = "normal"


In [5]:
curve_rate = 0.06
kappa = 0.03
swaption_expiry=5
swap_maturity = 10

initial_curve = get_mock_yield_curve_const(rate=curve_rate)


loca_vola = LinearLocalVolatility.from_const(swap_maturity, 0.4, 0.01, 0.1)

swap_pricer = SwapPricer(initial_curve, kappa)
swap = Swap(swaption_expiry, swap_maturity, 0.5)
swaption_pricer = SwaptionPricer(swap_pricer)


In [6]:
# Simulating XY processes

bond_measure = swap.bond_T0

process_simulator = ProcessSimulatorTerminalMeasure(number_paths, number_time_steps,
                                             swaption_expiry / number_time_steps,
                                             random_number_generator_type, bond_measure,
                                                    swap_pricer.bond_pricer, nr_processes=6,
                                                    n_scrambles=64)

# able to do multiprocessing in Python
result_obj = process_simulator.simulate_xy(kappa, loca_vola, parallel_simulation=True)


In [10]:
# Defining coupon grid for swaptions
coupon_grid = [0, +0.0025, -0.0025, +0.005, -0.005, +0.01, -0.01, 0.015, -0.015, 0.02, -0.02]

atm_swap_price = swap_pricer.price(swap, 0, 0, 0)
strike_grid = [atm_swap_price+coupon for coupon in coupon_grid]


In [11]:
## Analyzing resuts

def calculate_implied_vola():    
    output_all_df_ls = []
    for strike in strike_grid:
        swaption = Swaption(swaption_expiry, strike, swap)

        swaption_value_paths = monte_carlo_pricer_terminal_measure(result_obj, swaption, swaption_pricer)
        last_mc_time = result_obj.time_grid[-1]
        
        # Use Annuity as control variate instrument
        swaption_value_paths_cv2 = apply_control_variate_annuity(last_mc_time, result_obj.x[:, -1],
                                                        result_obj.y[:, -1], swaption_value_paths,
                                                        swap.annuity, swap_pricer.annuity_pricer,
                                                         swap_pricer.annuity_pricer.bond_pricer.initial_curve)

        swaption_value_mean = swaption_value_paths.mean()
        std, swaption_value_error = result_obj.calculate_std_error(swaption_value_paths, result_obj.n_scrambles)

        # swaption_value_mean_cv = swaption_value_paths_cv.mean()
        # std, swaption_value_error_cv = result_obj.calculate_std_error(swaption_value_paths_cv, result_obj.n_scrambles)

        swaption_value_mean_cv = swaption_value_paths_cv2.mean()
        std, swaption_value_error_cv = result_obj.calculate_std_error(swaption_value_paths_cv2,
                                                                      result_obj.n_scrambles)

        bond_pricer = swap_pricer.bond_pricer
        output_data = {"number_paths": number_paths, "number_time_steps": number_time_steps,
                       "random_number_generator_type": random_number_generator_type, "expiry": swaption_expiry,
                       "maturity": swap_maturity, "strike": strike, "atm strike": atm_swap_price,
                       "moneyness": strike-atm_swap_price,
                       "curve_rate": curve_rate, "kappa": kappa, "swaption value": swaption_value_mean,
                       "swaption value error": swaption_value_error,
                       "swaption value cv": swaption_value_mean_cv,
                       "swaption value error cv": swaption_value_error_cv,

                       "implied_vola": find_implied_black_vola(swaption_value_mean, swaption,
                                                               swap_pricer, bond_pricer),
                       "implied_vola_max": find_implied_black_vola(swaption_value_mean+swaption_value_error,
                                                                   swaption, swap_pricer, bond_pricer),
                       "implied_vola_min": find_implied_black_vola(swaption_value_mean-swaption_value_error,
                                                                   swaption, swap_pricer, bond_pricer),
                       "implied_vola_cv": find_implied_black_vola(swaption_value_mean_cv, swaption,
                                                               swap_pricer, bond_pricer),
                       "implied_vola_cv_max": find_implied_black_vola(swaption_value_mean_cv + swaption_value_error_cv, swaption,
                                                               swap_pricer, bond_pricer),
                       "implied_vola_cv_min": find_implied_black_vola(swaption_value_mean_cv - swaption_value_error_cv, swaption,
                                                               swap_pricer, bond_pricer)}

        output_df_new = pd.DataFrame(output_data, index=[0])
        output_all_df_ls.append(output_df_new)

    return output_all_df_ls


In [12]:
output_all_df_ls = calculate_implied_vola()
output_all_df = pd.concat(output_all_df_ls)
output_all_df.sort_values("moneyness")

C:\Users\chmou\Anaconda3\envs\quassi_gaussian\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


,number_paths,number_time_steps,random_number_generator_type,expiry,maturity,strike,atm strike,moneyness,curve_rate,kappa,swaption value,swaption value error,swaption value cv,swaption value error cv,implied_vola,implied_vola_max,implied_vola_min,implied_vola_cv,implied_vola_cv_max,implied_vola_cv_min
0,16384,1024,normal,5,10,0.040909,0.060909,-0.0200,0.06,0.03,0.063069,0.000507,0.063059,0.000008,0.065309,0.100121,0.012175,0.061337,0.064312,0.056097
0,16384,1024,normal,5,10,0.045909,0.060909,-0.0150,0.06,0.03,0.047463,0.000508,0.047452,0.000026,0.062064,0.079083,0.015049,0.061476,0.062887,0.059912
0,16384,1024,normal,5,10,0.050909,0.060909,-0.0100,0.06,0.03,0.032572,0.000494,0.032562,0.000063,0.060904,0.067794,0.052246,0.060749,0.061695,0.059776
0,16384,1024,normal,5,10,0.055909,0.060909,-0.0050,0.06,0.03,0.019646,0.000423,0.019638,0.000113,0.059856,0.062991,0.056652,0.059792,0.060640,0.058938
0,16384,1024,normal,5,10,0.058409,0.060909,-0.0025,0.06,0.03,0.014383,0.000371,0.014376,0.000136,0.059362,0.061688,0.057027,0.059316,0.060168,0.058463
0,16384,1024,normal,5,10,0.060909,0.060909,0.0000,0.06,0.03,0.010111,0.000314,0.010105,0.000146,0.059076,0.060912,0.057241,0.059041,0.059896,0.058187
0,16384,1024,normal,5,10,0.063409,0.060909,0.0025,0.06,0.03,0.006776,0.000252,0.006771,0.000142,0.058617,0.060131,0.057100,0.058590,0.059442,0.057736
0,16384,1024,normal,5,10,0.065909,0.060909,0.0050,0.06,0.03,0.004361,0.000193,0.004358,0.000127,0.058376,0.059676,0.057065,0.058353,0.059210,0.057491
0,16384,1024,normal,5,10,0.070909,0.060909,0.0100,0.06,0.03,0.001581,0.000099,0.001579,0.000086,0.057864,0.058926,0.056776,0.057848,0.058766,0.056909
0,16384,1024,normal,5,10,0.075909,0.060909,0.0150,0.06,0.03,0.000473,0.000056,0.000473,0.000053,0.057159,0.058411,0.055821,0.057146,0.058340,0.055873
